In [5]:
import pandas as pd
import numpy as np
import requests

## Get the time series for the following cryptocurrencies from cryptowat.ch starting from 2021-11-22, hourly data

    ETH
    SOL
    AVAX
    USDT
    FLOW


In [70]:
def get_data(token):
    
    if token == 'FLOW':
        exchange = 'kraken'
    else:
        exchange = 'coinbase-pro'
    res = requests.get(
        f'https://api.cryptowat.ch/markets/{exchange}/{token}usd/ohlc',
        params={
            'periods': '3600',
            'after': str(int(pd.Timestamp('2021-11-22').timestamp()))
        }
    )

    df = pd.DataFrame(
        res.json()['result']['3600'],
        columns=['ts', 'open', 'high', 'low', 'close', 'volume', 'volumeUSD']
    )
    df['ts'] = pd.to_datetime(df.ts, unit='s')
    df['token'] = token
    
    return df

In [71]:
tokens = ['ETH', 'SOL','AVAX', 'USDT', 'FLOW']

In [72]:
df_base = pd.concat(get_data(token) for token in tokens)


In [86]:
df = df_base.set_index('ts')

In [87]:
df.head()

,open,high,low,close,volume,volumeUSD,token
ts,,,,,,,
2021-11-22 00:00:00,4317.98,4342.24,4246.07,4262.99,7262.562789,3.123077e+07,ETH
2021-11-22 01:00:00,4263.04,4270.34,4212.45,4234.37,8437.946084,3.574586e+07,ETH
2021-11-22 02:00:00,4234.87,4246.72,4171.17,4217.89,9259.725370,3.895639e+07,ETH
2021-11-22 03:00:00,4217.88,4223.48,4163.58,4193.47,9259.899519,3.882032e+07,ETH
2021-11-22 04:00:00,4192.95,4213.59,4147.00,4168.35,7934.546906,3.315476e+07,ETH


In [88]:
df['token'].value_counts()


ETH     533
SOL     533
AVAX    533
USDT    533
FLOW    533
Name: token, dtype: int64

## Get the total USD volume traded for each token in a dataframe, sorted from highest volume to lowest volume

In [89]:
df.groupby('token')['volumeUSD'].sum().to_frame()

,volumeUSD
token,
AVAX,3.402515e+09
ETH,2.086614e+10
FLOW,8.528089e+07
SOL,6.824271e+09
USDT,1.956472e+09


## Add a column that calculates the close price ratio between ETH and SOL for each house (i.e. close price of ETH / close price of SOL for each period)

In [90]:
df2 = df.loc[df['token'] == 'ETH']['close'].rename('close_ETH').to_frame().join(
    df.loc[df['token'] == 'SOL']['close'].rename('close_SOL').to_frame()
)
df2['ETH_SOL_CLOSE_RATIO'] = df2['close_ETH'] / df2['close_SOL']

df = df.join(df2['ETH_SOL_CLOSE_RATIO'].to_frame())
del df2
df

,open,high,low,close,volume,volumeUSD,token,ETH_SOL_CLOSE_RATIO
ts,,,,,,,,
2021-11-22 00:00:00,4317.9800,4342.2400,4246.0700,4262.9900,7.262563e+03,3.123077e+07,ETH,18.471694
2021-11-22 00:00:00,230.4830,234.6880,228.8050,230.7850,9.353180e+04,2.172284e+07,SOL,18.471694
2021-11-22 00:00:00,131.6900,133.7900,127.7000,128.6700,9.686748e+04,1.267080e+07,AVAX,18.471694
2021-11-22 00:00:00,1.0010,1.0017,1.0009,1.0012,9.987681e+06,9.999090e+06,USDT,18.471694
2021-11-22 00:00:00,13.8870,14.1690,13.8870,14.1500,2.326138e+04,3.274755e+05,FLOW,18.471694
...,...,...,...,...,...,...,...,...
2021-12-14 04:00:00,3781.7600,3800.4600,3774.9500,3776.2100,2.258430e+03,8.557043e+06,ETH,24.525622
2021-12-14 04:00:00,154.7400,154.8800,153.2900,153.9700,2.692133e+04,4.152349e+06,SOL,24.525622
2021-12-14 04:00:00,78.9600,79.4900,78.5100,78.6600,2.137019e+04,1.691930e+06,AVAX,24.525622


## Change the name of the volume and volumeUSD columns to volumeBase and volumeTerm

In [94]:
df = df.rename(
    columns={
        'volume':'volumeBase',
        'volumeUSD':'volumeTerm'
    }
)
df

,open,high,low,close,volumeBase,volumeTerm,token,ETH_SOL_CLOSE_RATIO
ts,,,,,,,,
2021-11-22 00:00:00,4317.9800,4342.2400,4246.0700,4262.9900,7.262563e+03,3.123077e+07,ETH,18.471694
2021-11-22 00:00:00,230.4830,234.6880,228.8050,230.7850,9.353180e+04,2.172284e+07,SOL,18.471694
2021-11-22 00:00:00,131.6900,133.7900,127.7000,128.6700,9.686748e+04,1.267080e+07,AVAX,18.471694
2021-11-22 00:00:00,1.0010,1.0017,1.0009,1.0012,9.987681e+06,9.999090e+06,USDT,18.471694
2021-11-22 00:00:00,13.8870,14.1690,13.8870,14.1500,2.326138e+04,3.274755e+05,FLOW,18.471694
...,...,...,...,...,...,...,...,...
2021-12-14 04:00:00,3781.7600,3800.4600,3774.9500,3776.2100,2.258430e+03,8.557043e+06,ETH,24.525622
2021-12-14 04:00:00,154.7400,154.8800,153.2900,153.9700,2.692133e+04,4.152349e+06,SOL,24.525622
2021-12-14 04:00:00,78.9600,79.4900,78.5100,78.6600,2.137019e+04,1.691930e+06,AVAX,24.525622


In [92]:
df

,open,high,low,close,volume,volumeUSD,token,ETH_SOL_CLOSE_RATIO
ts,,,,,,,,
2021-11-22 00:00:00,4317.9800,4342.2400,4246.0700,4262.9900,7.262563e+03,3.123077e+07,ETH,18.471694
2021-11-22 00:00:00,230.4830,234.6880,228.8050,230.7850,9.353180e+04,2.172284e+07,SOL,18.471694
2021-11-22 00:00:00,131.6900,133.7900,127.7000,128.6700,9.686748e+04,1.267080e+07,AVAX,18.471694
2021-11-22 00:00:00,1.0010,1.0017,1.0009,1.0012,9.987681e+06,9.999090e+06,USDT,18.471694
2021-11-22 00:00:00,13.8870,14.1690,13.8870,14.1500,2.326138e+04,3.274755e+05,FLOW,18.471694
...,...,...,...,...,...,...,...,...
2021-12-14 04:00:00,3781.7600,3800.4600,3774.9500,3776.2100,2.258430e+03,8.557043e+06,ETH,24.525622
2021-12-14 04:00:00,154.7400,154.8800,153.2900,153.9700,2.692133e+04,4.152349e+06,SOL,24.525622
2021-12-14 04:00:00,78.9600,79.4900,78.5100,78.6600,2.137019e+04,1.691930e+06,AVAX,24.525622


## create a fat table indexed by the timestamp, and each column is the close price of each token (i.e. this should be a table of 200 rows and 5 columns)

In [110]:
fat_table = df.loc[df['token'] == 'ETH']['close'].rename('close_ETH').to_frame().join(
    df.loc[df['token'] == 'SOL']['close'].rename('close_SOL').to_frame()).join(
    df.loc[df['token'] == 'AVAX']['close'].rename('close_AVAX').to_frame()).join(
    df.loc[df['token'] == 'USDT']['close'].rename('close_USDT').to_frame()).join(
    df.loc[df['token'] == 'FLOW']['close'].rename('close_FLOW').to_frame())
fat_table.head()


,close_ETH,close_SOL,close_AVAX,close_USDT,close_FLOW
ts,,,,,
2021-11-22 00:00:00,4262.99,230.785,128.67,1.0012,14.150
2021-11-22 01:00:00,4234.37,226.000,134.85,1.0013,13.621
2021-11-22 02:00:00,4217.89,227.037,132.54,1.0012,13.753
2021-11-22 03:00:00,4193.47,223.117,133.38,1.0011,13.652
2021-11-22 04:00:00,4168.35,220.652,136.29,1.0009,13.581


## calculate the hour by hour log return of the close price of each token (return is calculated by np.log(price_t / price_{t-1}))

In [112]:
fat_table['ETH_LOG_RETURN'] = np.log(fat_table['close_ETH'] / fat_table['close_ETH'].shift(1))
fat_table['SOL_LOG_RETURN'] = np.log(fat_table['close_SOL'] / fat_table['close_SOL'].shift(1))
fat_table['AVAX_LOG_RETURN'] = np.log(fat_table['close_AVAX'] / fat_table['close_AVAX'].shift(1))
fat_table['USDT_LOG_RETURN'] = np.log(fat_table['close_USDT'] / fat_table['close_USDT'].shift(1))
fat_table['FLOW_LOG_RETURN'] = np.log(fat_table['close_FLOW'] / fat_table['close_FLOW'].shift(1))
fat_table.head()

,close_ETH,close_SOL,close_AVAX,close_USDT,close_FLOW,ETH_LOG_RETURN,SOL_LOG_RETURN,AVAX_LOG_RETURN,USDT_LOG_RETURN,FLOW_LOG_RETURN
ts,,,,,,,,,,
2021-11-22 00:00:00,4262.99,230.785,128.67,1.0012,14.150,NaN,NaN,NaN,NaN,NaN
2021-11-22 01:00:00,4234.37,226.000,134.85,1.0013,13.621,-0.006736,-0.020952,0.046912,0.0001,-0.038102
2021-11-22 02:00:00,4217.89,227.037,132.54,1.0012,13.753,-0.003900,0.004578,-0.017279,-0.0001,0.009644
2021-11-22 03:00:00,4193.47,223.117,133.38,1.0011,13.652,-0.005806,-0.017417,0.006318,-0.0001,-0.007371
2021-11-22 04:00:00,4168.35,220.652,136.29,1.0009,13.581,-0.006008,-0.011109,0.021583,-0.0002,-0.005214


In [120]:
corr_table = fat_table[['ETH_LOG_RETURN','SOL_LOG_RETURN','AVAX_LOG_RETURN','USDT_LOG_RETURN','FLOW_LOG_RETURN']].corr()
corr_table

,ETH_LOG_RETURN,SOL_LOG_RETURN,AVAX_LOG_RETURN,USDT_LOG_RETURN,FLOW_LOG_RETURN
ETH_LOG_RETURN,1.000000,0.737801,0.644433,-0.118869,0.562097
SOL_LOG_RETURN,0.737801,1.000000,0.622500,0.106857,0.445335
AVAX_LOG_RETURN,0.644433,0.622500,1.000000,-0.017875,0.389020
USDT_LOG_RETURN,-0.118869,0.106857,-0.017875,1.000000,-0.196579
FLOW_LOG_RETURN,0.562097,0.445335,0.389020,-0.196579,1.000000


In [123]:
corr_table.style.background_gradient(cmap='coolwarm')

,ETH_LOG_RETURN,SOL_LOG_RETURN,AVAX_LOG_RETURN,USDT_LOG_RETURN,FLOW_LOG_RETURN
ETH_LOG_RETURN,1.000000,0.737801,0.644433,-0.118869,0.562097
SOL_LOG_RETURN,0.737801,1.000000,0.622500,0.106857,0.445335
AVAX_LOG_RETURN,0.644433,0.622500,1.000000,-0.017875,0.389020
USDT_LOG_RETURN,-0.118869,0.106857,-0.017875,1.000000,-0.196579
FLOW_LOG_RETURN,0.562097,0.445335,0.389020,-0.196579,1.000000
